In [68]:
%run model_sucrose.ipynb

Knocking out reactions mentioned in the paper by Wu et al (Wu et al., 2019):
- LDH which converts pyruvate to lactate
- PTA which converts Acetyl-CoA to Acetyl phosphate

### Finding knockout targets

#### Finding the LDH reaction:

In [69]:
model.reactions.query("LDH")

[<Reaction BETALDHx at 0x1ca5e3cdf70>,
 <Reaction LDH2 at 0x1ca5e66ddf0>,
 <Reaction LDH_L at 0x1ca5e680ee0>,
 <Reaction VALDHr at 0x1ca5e8e6d60>]

Checking if LDH_L is the one producing lactate

In [70]:
model.reactions.LDH_L.metabolites

{<Metabolite lac__L_c at 0x1ca5e075280>: -1.0,
 <Metabolite nad_c at 0x1ca5e07e700>: -1.0,
 <Metabolite h_c at 0x1ca5e0706a0>: 1.0,
 <Metabolite nadh_c at 0x1ca5e07e940>: 1.0,
 <Metabolite pyr_c at 0x1ca5e08cd00>: 1.0}

In [71]:
model.metabolites.lac__L_c.name

'L-Lactate'

Yes. Seems like LDH_L is the reaction we're looking for.

#### Finding the PTA reaction

In [72]:
model.reactions.query("PTA")

[<Reaction APTA1i at 0x1ca5e330d30>,
 <Reaction ASPTA at 0x1ca5e3cdc70>,
 <Reaction GF6PTA at 0x1ca5e50a910>,
 <Reaction PTAr at 0x1ca5e729eb0>,
 <Reaction PTA2 at 0x1ca5e729d00>]

Checking if PTAr is the reaction Acetyl phosphate:

In [73]:
model.reactions.PTAr.metabolites

{<Metabolite accoa_c at 0x1ca5e0311f0>: -1.0,
 <Metabolite pi_c at 0x1ca5e07a9d0>: -1.0,
 <Metabolite actp_c at 0x1ca5e03c850>: 1.0,
 <Metabolite coa_c at 0x1ca5e01d460>: 1.0}

In [74]:
model.metabolites.actp_c.name

'Acetyl phosphate'

Yes. Seems like PTAr is the reaction we're looking for.

### Knocking out reactions

Now we'll try knocking it out each of the reactions and see what happens to the growth and surfactin yield:

Knockout of lac__L_c:

In [97]:
with model:
    model.objective = model.reactions.SP_1
    model.reactions.LDH_L.bounds = 0, 0
    print("ATPPRT's bounds inside the with statement")
    print(model.reactions.LDH_L.lower_bound, model.reactions.LDH_L.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    with model:
        model.objective = model.reactions.SP_1
        surfactin_production = model.optimize().objective_value
        print("Max. succinate production [mmol gDW^-1 h^-1]:", surfactin_production)
        print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))

ATPPRT's bounds inside the with statement
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.3436433767512154
Max. succinate production [mmol gDW^-1 h^-1]: 0.3436433767512154
Theoretical max. yield [mmol-succ / mmol-ac]: 0.12001049195903496


Knockout of PTAr:

In [98]:
with model:
    model.objective = model.reactions.SP_1
    model.reactions.PTAr.bounds = 0, 0
    print("ATPPRT's bounds inside the with statement")
    print(model.reactions.PTAr.lower_bound, model.reactions.PTAr.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    with model:
        model.objective = model.reactions.SP_1
        surfactin_production = model.optimize().objective_value
        print("Max. succinate production [mmol gDW^-1 h^-1]:", surfactin_production)
        print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))

ATPPRT's bounds inside the with statement
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.3436433767512154
Max. succinate production [mmol gDW^-1 h^-1]: 0.3436433767512154
Theoretical max. yield [mmol-succ / mmol-ac]: 0.11996699542671414


In both cases we see that the yield get's a little higher. 